# 🚀 Analyse 2: Keyword in Context (KWIC) 

<div class="alert alert-block alert-info"> <b> 🔔 Feinlernziel(e) dieses Kapitels</b></br>
   Sie können die Darstellungsmethode <i>Keywords in Context</i> beschreiben, Wörter zur Anzeige auswählen und diese anzeigen lassen.</div>

## Hinweise zur Ausführung des Notebooks
Dieses **Notebook** kann auf unterschiedlichen Levels erarbeitet werden (siehe Abschnitt ["Technische Voraussetzungen"](../markdown/introduction_requirements)): 
1. Book-Only Mode
2. Cloud Mode: Dafür auf 🚀 klicken und z.B. in Colab ausführen.
3. Local Mode: Dafür auf Herunterladen ↓ klicken und ".ipynb" wählen. 

## Übersicht 
Im Folgenden werden die annotierten Dateien (CSV-Format) analysiert. Unser Ziel ist es, die Wort-/Lemma-Häufigkeiten einer vordefinierten Wortgruppe für die Monate des Jahres 1918 zu plotten und zu sehen, ob sie mit den Wellen der Grippepandemie korrelieren.
Dafür werden folgendene Schritte durchgeführt:
1. Einlesen des Korpus, der Metadaten und der Grippe-Wortliste
2. Extraktion und Darstellung der Wortkontexte durch KWIC

<details>
  <summary><b>Informationen zum Ausführen des Notebooks – Zum Ausklappen klicken ⬇️</b></summary>
  
<b>Voraussetzungen zur Ausführung des Jupyter Notebooks</b>
<ol>
<li> Installieren der Bibliotheken </li>
<li> Pfad zu den Daten setzen</li>
<li> Laden der Daten (z.B. über den Command `wget` (s.u.))</li>
</ol>
Zum Testen: Ausführen der Zelle "load libraries" und der Sektion "Einlesen der Daten". </br>
Alle Zellen, die mit 🚀 gekennzeichnet sind, werden nur bei der Ausführung des Noteboos in Colab / JupyterHub bzw. lokal ausgeführt. 
</details>

In [ ]:
#  🚀 Install libraries 
! pip install pandas bokeh 

In [4]:
import re
from pathlib import Path
import pandas as pd

## for interactivity in jupyter books
from bokeh.io import output_notebook, show
from bokeh.layouts import column
from bokeh.models import CustomJS, TextInput, Div
# Ensure Bokeh output is displayed in the notebook
output_notebook()

Loading BokehJS ...

## 1. Einlesen der Daten, Metadaten

### 1.1 Einlesen des Korpus (CSV-Dateien)

<details>
  <summary><b>Informationen zum Ausführen des Notebooks – Zum Ausklappen klicken ⬇️</b></summary>
Zuerst wird der Ordner angelegt, in dem die CSV-Dateien gespeichert werden. Der Einfachheit halber wird die gleich Datenablagestruktur wie in dem <a href="https://github.com/dh-network/quadriga/tree/main">GitHub Repository</a>, in dem die Daten gespeichert sind, vorausgesetzt. </br>
Danach werden alle CSV-Dateien im Korpus heruntergeladen und gespeichert. Dafür sind folgende Schritte nötig:
<ol>
    <li>Es wird eine Liste erstellt, die die URLs zu den einzelnen CSV-Dateien beinhaltet.</li>
    <li>Die Liste wird als txt-Datei gespeichert.</li>
    <li>Alle Dateien aus der Liste werden heruntergeladen und in dem Ordner <i>../data/csv</i> gespeichert.</li>
</ol>
Sollten die Dateien schon an einem anderen Ort vorhanden sein, können die Dateipfade zu den Ordnern angepasst werden. </br>
</details>

In [ ]:
# 🚀 Create data directory path
corpus_dir = Path("../data/csv")
if not corpus_dir.exists():
    corpus_dir.mkdir()

In [ ]:
# 🚀 Create download list 
github_api_txt_dir_path = "https://api.github.com/repos/dh-network/quadriga/contents/data/csv"
txt_dir_info = requests.get(github_api_txt_dir_path).json()
url_list = [entry["download_url"] for entry in txt_dir_info]

# 🚀 Write download list as txt file
url_list_path = Path("github_csv_file_urls.txt")
with url_list_path.open('w') as output_txt:
    output_txt.write("\n".join(url_list))

In [ ]:
# ⚠️ Only execute, if you haven't downloaded the files yet!
# 🚀 Download all csv files – this step will take a while (ca. 7 minutes)
! wget -i github_csv_file_urls.txt -P ../data/csv

Setzen des Pfads:

In [5]:
# set the path to csv files to be processed
csv_dir = Path(r"../data/csv")

In [6]:
# Create dictionary to save the corpus data (filenames and tables)
corpus_annotations = {}

# Iterate over csv files 
for file in csv_dir.iterdir():
    # check if the entry is a file, not a directory
    if file.is_file():
        # check if the file has the correct suffix csv
        if file.suffix == '.csv':
            # read the csv table to a data frame
            data = pd.read_csv(file) 
            # save the data frame to the dictionary, key=filename (without suffix), value=dataframe
            corpus_annotations[file.name] = data

### 1.2 Einlesen der Metadaten

<details>
  <summary><b>Informationen zum Ausführen des Notebooks – Zum Ausklappen klicken ⬇️</b></summary>
Zuerst wird der Ordner angelegt, in dem die Metadaten-Datei gespeichert wird. Wieder wird die gleich Datenablagestruktur wie in dem <a href="https://github.com/dh-network/quadriga/tree/main">GitHub Repository</a> vorausgesetzt. </br>
Der Text wird aus GitHub heruntergeladen und in dem Ordner <i>../data/metadata/</i> abgespeichert. </br>
Der Pfad kann in der Variable <i>metadata_path</i> angepasst werden. Die einzulesende Datei muss die Endung `.csv` haben. </br>
</details>

In [ ]:
# 🚀 Create metadata directory path
metadata_dir = Path("../data/metadata")
if not metadata_dir.exists():
    metadata_dir.mkdir()

In [ ]:
# 🚀 Load the metadata file from GitHub 
! wget https://raw.githubusercontent.com/dh-network/quadriga/refs/heads/main/data/metadata/QUADRIGA_FS-Text-01_Data01_Corpus-Table.csv -P ../data/metadata

In [7]:
# set path to metadata file
metadata_path = '../data/metadata/QUADRIGA_FS-Text-01_Data01_Corpus-Table.csv'

# read metadata file to pandas dataframe and set index
corpus_metadata = pd.read_csv(metadata_path, sep=';')
corpus_metadata = corpus_metadata.set_index('DC.identifier')

## 2.  Extraktion und Darstellung der Wortkontexte durch KWIC

In [9]:
class ContextViewer:
    
    def __init__(self, corpus_annotated, corpus_metadata):
        self.prepare_index_dataframe_for_search(corpus_annotated, corpus_metadata)
    
    def prepare_index_dataframe_for_search(self, corpus_annotated, corpus_metadata):
        for filename, annotated_text in corpus_annotated.items():
            txtname = filename.replace('.csv', '')
            if txtname in corpus_metadata.index:
                year, month, day = self.get_date_fname(txtname, corpus_metadata)
                annotated_text['month'] = month
                annotated_text['filename'] = filename
        self.full_df = pd.concat(corpus_annotated.values())
        self.full_df = self.full_df.reset_index()
        print(f'Searching in a corpus of {self.full_df.shape[0]} word occurences')
        
    def get_date_fname(self, txtname, corpus_metadata):  
        date = corpus_metadata.loc[txtname, 'DC.date']
        date = str(date)
        year = date[:4]
        month = date[:7]
        day = date
        return year, month, day 
    
    def get_context_words(self, search_terms, n_words):
        #search_terms = input('Insert a word to search, split by comma if more than one: ')
        if len(search_terms) == 0:
            search_terms = 'Grippe, Krankheit'
        search_terms = search_terms.split(',')
        search_terms = [x.strip() for x in search_terms]
        indices = self.full_df.query(f'Lemma.isin({search_terms})').index
        #print(indices)
        left_contexts = []
        this_words = []
        right_contexts = []
        months = []
        for indice in indices:
            left = self.full_df.iloc[indice-10:indice-1, ]["Token"]
            leftс = left[~left.str.contains('\n')]
            right = self.full_df.iloc[indice+1:indice+10, ]["Token"]
            rightс = right[~right.str.contains('\n')]
            left_contexts.append(' '.join(leftс))
            right_contexts.append(' '.join(rightс))
            this_words.append(self.full_df.iloc[indice, ]["Token"])
            months.append(self.full_df.iloc[indice, ]["month"])
        newdf = pd.DataFrame()
        newdf['left_context'] = left_contexts
        newdf['word'] = this_words
        newdf['right_context'] = right_contexts
        newdf['month'] = months
        return newdf
        
    ## currently unused functionality:
    def get_context_sents(self, n_sentences):
        search_lemma = input('Insert a word to search: ')
        if len(search_lemma) == 0:
            search_lemma = 'Grippe'
        indices = self.full_df.query(f'Lemma=="{search_lemma}"').index
        #print(indices)
        left_contexts = []
        this_sentences = []
        right_contexts = []
        months = []
        for indice in indices:
            #print(indice)
            current_filename = self.full_df.iloc[indice, ]["filename"]
            current_sentence_id = self.full_df.iloc[indice, ]["Sentence_idx"]
            left_context = self.get_sents(direction=-1, 
                                              current_filename=current_filename, 
                                              current_sentence_id=current_sentence_id, 
                                              n_sentences=n_sentences) 
            left_contexts.append(left_context)
            right_context = self.get_sents(direction=1, 
                                               current_filename=current_filename, 
                                               current_sentence_id=current_sentence_id, 
                                               n_sentences=n_sentences) 
            right_contexts.append(right_context)
            this_sentence = self.get_sents(direction=0, 
                                               current_filename=current_filename,
                                               current_sentence_id=current_sentence_id,
                                               n_sentences=1)
            this_sentences.append(this_sentence)
            #this_words.append(self.full_df.iloc[indice, ]["Token"])
            months.append(self.full_df.iloc[indice, ]["month"])
        newdf = pd.DataFrame()
        newdf['left_sentences'] = left_contexts
        newdf['this_sentence'] = this_sentences
        newdf['right_sentences'] = right_contexts
        newdf['month'] = months
        return newdf #.sort_values(by='month')
    
    def get_sents(self, direction, current_filename, current_sentence_id, n_sentences):
        sentences = []
        for n in range(1,n_sentences+1):
            sentence_id = current_sentence_id + (n * direction)
            this_sentence = self.create_sentence(current_filename, sentence_id)
            sentences.append(this_sentence)
        #print(' '.join(sentences))
        return ' '.join(sentences)
    
    def create_sentence(self, current_filename, sentence_id):
        words = self.full_df.query(f'filename=="{current_filename}" and Sentence_idx=={sentence_id}')['Token']
        sentence = ' '.join(words)
        #print(sentence)
        return sentence
        

In [10]:
search_terms = TextInput(value='Grippe, Krankheit', 
                                 title="Geben Sie die zu suchenden Wörter ein und trennen Sie sie durch Kommas, wenn es mehrere sind:") #input('Insert words to search, split by comma if more than one: ')

search_terms_str = search_terms.value.strip()
# JavaScript callback to update the in Jupyter Book
rewrite_var_after_input = CustomJS(args=dict(text_input=search_terms), code="""
    var word = text_input.value.trim();
    console.log('Input value:', word);
    function sendToPython(){
    var kernel = IPython.notebook.kernel;
    kernel.execute("search_terms_str = '" + word + "'");
    }
    sendToPython();
""")



search_terms.js_on_change('value', rewrite_var_after_input)

# Layout and display
layout = column(search_terms)

show(layout)

In [ ]:
kwic = ContextViewer(corpus_annotations, corpus_metadata)

In [ ]:
kwic.get_context_words(search_terms_str, n_words=5)